In [1]:
import splitfolders

In [2]:
source_path='C:/armaan/Machine Learning/Datasets/Real Life Violence Dataset'
destination_path='C:/armaan/Machine Learning/Datasets/violence splitted'

In [3]:
split_ratio=(0.75,0.15,0.1)
splitfolders.ratio(source_path,destination_path,seed=42,ratio=split_ratio,group_prefix=None)

Copying files: 2000 files [00:08, 247.24 files/s]


In [4]:
import cv2
import os
from tqdm import tqdm

In [23]:
input_folder='C:/armaan/Machine Learning/Datasets/violence splitted/val'
output_folder='C:/armaan/Machine Learning/Datasets/frames/val'
categories=['Violence','NonViolence']

In [19]:
def get_frames(video_path,output_folder,frame_rate=1):
    os.makedirs(output_folder,exist_ok=True)
    video_capture=cv2.VideoCapture(video_path)
    success,image=video_capture.read()
    count=0
    file_name_count=1
    # fps = int(video_capture.get(cv2.CAP_PROP_FPS))  # Get the frames per second (FPS) of the video

    while success:
        if count%5==0:
            frame_file_name=os.path.join(output_folder,f"frame{file_name_count}.jpg")
            cv2.imwrite(frame_file_name,image)
            file_name_count+=1
        success,image=video_capture.read()
        count+=1
    video_capture.release()


In [ ]:
def process_videos(input_folder,output_folder,categories,frame_rate=1):
    for category in categories:
        category_input_path=os.path.join(input_folder,category)
        category_output_path=os.path.join(output_folder,category)
        
        for video_file in tqdm(os.listdir(category_input_path)):
            video_path=os.path.join(category_input_path,video_file)
            video_name,_=os.path.splitext(video_file)
            video_output_folder = os.path.join(category_output_path, video_name)
            get_frames(video_path,video_output_folder,frame_rate)

In [24]:
process_videos(input_folder,output_folder,categories)

100%|██████████| 150/150 [00:44<00:00,  3.35it/s]


In [1]:

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import os

c:\Users\Armaan\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from tqdm import tqdm
from tensorflow.keras.models import Model

In [3]:
mobile_net_model = tf.keras.applications.MobileNetV2()
mobile_net_model=Model(
    inputs=mobile_net_model.inputs,outputs=mobile_net_model.layers[-2].output
)

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [4]:
for layers in mobile_net_model.layers:
    layers.trainable=False

In [5]:
def extract_features(frame_path,model):
    img=image.load_img(frame_path,target_size=(224,224))
    img_array=image.img_to_array(img)
    img_array=np.expand_dims(img_array,axis=0)
    img_array=preprocess_input(img_array)
    features=model.predict(img_array,verbose=False)
    return features.flatten()

In [6]:
#testing the mobilenet model 
temp=extract_features('C:/armaan/Machine Learning/Datasets/frames/train/NonViolence/NV_4/frame1.jpg',mobile_net_model)
print(temp)
print(len(temp))

[0.15216883 0.33164126 0.21129113 ... 1.4389979  0.         0.07342909]
1280


In [9]:
def process_frames(frames_folder,model):
    labels=[]
    features=[]
    for category in os.listdir(frames_folder):
        category_path=os.path.join(frames_folder,category)
        for video_folder in tqdm(os.listdir(category_path)):
            video_path=os.path.join(category_path,video_folder)
            video_features=[]
            for frame in sorted(os.listdir(video_path)):
                frame_path=os.path.join(video_path,frame)
                curr_features=extract_features(frame_path,model)
                video_features.append(curr_features)
            features.append(video_features)
            labels.append(category)
    return features, labels

In [10]:
train_features,train_labels=process_frames('C:/armaan/Machine Learning/Datasets/frames/train',mobile_net_model)

100%|██████████| 750/750 [29:16<00:00,  2.34s/it]  


In [13]:
import pickle
with open('train_features1.pkl', 'wb') as f:
    pickle.dump(train_features, f)
    
with open('train_labels1.pkl', 'wb') as f:
    pickle.dump(train_labels, f)

In [12]:
val_features,val_labels=process_frames('C:/armaan/Machine Learning/Datasets/frames/val',mobile_net_model)

100%|██████████| 150/150 [10:37<00:00,  4.25s/it] 


In [14]:
with open('val_features1.pkl', 'wb') as f:
    pickle.dump(val_features, f)
    
with open('val_labels1.pkl', 'wb') as f:
    pickle.dump(val_labels, f)

In [16]:
test_features,test_labels=process_frames('C:/armaan/Machine Learning/Datasets/frames/test',mobile_net_model)

100%|██████████| 100/100 [05:15<00:00,  3.16s/it]


In [17]:
with open('test_features1.pkl', 'wb') as f:
    pickle.dump(test_features, f)
    
with open('test_labels1.pkl', 'wb') as f:
    pickle.dump(test_labels, f)